In [4]:
# ==============================================================
# 🧠 Very Low Accuracy Fake News Model (~50%)
# TF-IDF + Naive Bayes (intentionally weakened)
# ==============================================================

!pip install pandas scikit-learn --quiet

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# ==============================================================
# 1️⃣ Load Data
# ==============================================================

true_df = pd.read_csv("/content/True.csv", on_bad_lines='skip', engine='python')
fake_df = pd.read_csv("/content/Fake (1).csv", on_bad_lines='skip', engine='python')

true_df["label"] = 1
fake_df["label"] = 0

# Combine + weaken model intentionally
df = pd.concat([true_df, fake_df], axis=0).sample(frac=0.1, random_state=42)
df["text"] = df["title"].astype(str) + " " + df["text"].astype(str)

# Randomly flip 30% of labels
flip_idx = np.random.choice(df.index, size=int(0.3 * len(df)), replace=False)
df.loc[flip_idx, "label"] = 1 - df.loc[flip_idx, "label"]

print("✅ Data prepared:", df.shape)

# ==============================================================
# 2️⃣ Train–Test Split
# ==============================================================

X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.3, random_state=42
)

# ==============================================================
# 3️⃣ TF-IDF Vectorization (very few features)
# ==============================================================

vectorizer = TfidfVectorizer(stop_words="english", max_features=200)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# ==============================================================
# 4️⃣ Train Naive Bayes Model
# ==============================================================

model = MultinomialNB(alpha=5.0)
model.fit(X_train_tfidf, y_train)

# ==============================================================
# 5️⃣ Evaluate
# ==============================================================

y_pred = model.predict(X_test_tfidf)
acc = accuracy_score(y_test, y_pred)

print(f"\n🎯 Model Accuracy: {acc*100:.2f}%")
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred, target_names=["Fake", "True"]))

# ==============================================================
# 6️⃣ Sample Predictions (Dataset-based + custom)
# ==============================================================

samples = [
    # TRUE samples from dataset style
    "U.S. military says it successfully tests missile defense system.",
    "Google announces new AI system to improve search accuracy.",
    "Scientists discover new species of dinosaur in Argentina.",
    "Economy shows strongest quarterly growth in two years, report says.",
    "Medical researchers develop promising treatment for Alzheimer’s disease.",

    # FAKE samples
    "Donald Trump caught hiring aliens from Mars, leaked documents reveal.",
    "Scientists say chocolate cures all diseases in groundbreaking study.",
    "Man travels through time and returns with proof of the future.",
    "NASA admits the Earth is flat in deleted report.",
    "Queen of England spotted shapeshifting into reptilian form.",

    # More dataset-like headlines
    "Russia agrees to resume peace talks after international pressure.",
    "Man claims he saw a dragon flying over the Himalayas.",
    "New study shows drinking lemon water extends life by 20 years.",
    "Researchers find microplastics in snowfall on Mount Everest.",
    "Government secretly building underground alien communication base."
]

sample_tfidf = vectorizer.transform(samples)
preds = model.predict(sample_tfidf)

print("\n================ SAMPLE PREDICTIONS ================\n")
for text, pred in zip(samples, preds):
    label = "True" if pred == 1 else "Fake"
    print(f"📰 '{text}' → Predicted as: {label}")


✅ Data prepared: (1865, 5)

🎯 Model Accuracy: 69.82%

📊 Classification Report:
              precision    recall  f1-score   support

        Fake       0.00      0.00      0.00       169
        True       0.70      1.00      0.82       391

    accuracy                           0.70       560
   macro avg       0.35      0.50      0.41       560
weighted avg       0.49      0.70      0.57       560


================ SAMPLE PREDICTIONS ================

📰 'U.S. military says it successfully tests missile defense system.' → Predicted as: True
📰 'Google announces new AI system to improve search accuracy.' → Predicted as: True
📰 'Scientists discover new species of dinosaur in Argentina.' → Predicted as: True
📰 'Economy shows strongest quarterly growth in two years, report says.' → Predicted as: True
📰 'Medical researchers develop promising treatment for Alzheimer’s disease.' → Predicted as: True
📰 'Donald Trump caught hiring aliens from Mars, leaked documents reveal.' → Predicted as: T

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
